In [1]:
import pandas as pd

dataset_path = '/content/Cleaned_Lang8.csv'

df = pd.read_csv(dataset_path, quoting=3, encoding='utf-8', on_bad_lines = 'skip')
df.columns = ['Incorrect', 'Correct']
print(df.head())
print(df.info())


                                           Incorrect  \
0  the president was standing in the front row an...   
1                            i just learnt alphabet.   
2  i'm always not patient enough to take a self -...   
3  but it is also said that it is very difficult ...   
4           he is very humorious and handsome > _ <.   

                                             Correct  
0  the president was standing in the front row an...  
1                   i have just learnt the alphabet.  
2  i'm never patient enough to take a self - stud...  
3  but it is also said that it is very difficult ...  
4            he is very handsome and humorous > _ <.  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112726 entries, 0 to 112725
Data columns (total 2 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   Incorrect  112726 non-null  object
 1   Correct    112726 non-null  object
dtypes: object(2)
memory usage: 1.7+ MB
None


In [3]:
df = df.dropna()
df = df.drop_duplicates()
print({df.shape})



{(111212, 2)}


In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df['Incorrect'], df['Correct'], test_size=0.2, random_state= 42)


In [8]:
from transformers import T5Tokenizer
tokenizer = T5Tokenizer.from_pretrained('t5-small')
train_encodings = tokenizer(X_train.to_list(), truncation = True, padding = True, return_tensors = 'pt')
labels_encodings = tokenizer(y_train.to_list(), truncation = True, padding = True, return_tensors = 'pt')




In [9]:
from transformers import T5ForConditionalGeneration
model = T5ForConditionalGeneration.from_pretrained('t5-small')

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [10]:
import torch

class GrammarDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.encodings.input_ids)

    def __getitem__(self, idx):
        return {
            'input_ids': self.encodings.input_ids[idx],
            'attention_mask': self.encodings.attention_mask[idx],
            'labels': self.labels.input_ids[idx]
        }

train_dataset = GrammarDataset(train_encodings, labels_encodings)

In [13]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=2,
    per_device_train_batch_size=128,
    per_device_eval_batch_size=128,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    save_total_limit=2
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset
)

trainer.train()

Step,Training Loss
10,11.165900
20,11.254200
30,10.973300
40,10.621300
50,10.207700
60,9.720500
70,9.204800
80,8.360500
90,7.425900
100,6.680000


TrainOutput(global_step=1392, training_loss=1.0687987545783493, metrics={'train_runtime': 1468.2035, 'train_samples_per_second': 121.194, 'train_steps_per_second': 0.948, 'total_flos': 3104378252034048.0, 'train_loss': 1.0687987545783493, 'epoch': 2.0})

In [14]:
test_encodings = tokenizer(list(X_test), padding=True, truncation=True, return_tensors="pt")
test_dataset = GrammarDataset(test_encodings, tokenizer(list(y_test), padding=True, truncation=True, return_tensors="pt"))

trainer.evaluate(test_dataset)

{'eval_loss': 0.20624355971813202,
 'eval_runtime': 41.4381,
 'eval_samples_per_second': 536.777,
 'eval_steps_per_second': 4.199,
 'epoch': 2.0}

In [15]:
import torch

# Move model and inputs to the same device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Sample input text (use a longer input for better results)
sample_text = "I was take the subway to get to my academy on time"
input_ids = tokenizer(sample_text, return_tensors="pt", padding=True, truncation=True).input_ids.to(device)

# Generate corrected text with modified decoding parameters
try:
    outputs = model.generate(
        input_ids,
        max_length=50,
        num_beams=5,
        temperature=0.7,
        top_k=50,
        top_p=0.95,
        no_repeat_ngram_size=2
    )
    corrected_sentence = tokenizer.decode(outputs[0], skip_special_tokens=True)
    print("Corrected Sentence:", corrected_sentence)
except Exception as e:
    print("An error occurred during text generation:", e)

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:595: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


Corrected Sentence: I was taking the subway to get to my academy on time.
